In [24]:
import pandas as pd
import numpy as np
import json

from cleanup.cleanup_functions_collec import create_collec_staging
from cleanup.cleanup_siret_functions import clean_numeros

### Upload fichier

In [25]:
path = '../data/datasets/'

In [26]:
communities_df = pd.read_csv(path + 'all_communities_data.csv', delimiter=';', low_memory=False, index_col=False)

### Nettoyage de données

In [27]:
communities_clean = communities_df.copy()

In [28]:
communities_clean = create_collec_staging(communities_clean)

Cleaning duplicates: 36353 entries


After removing duplicates: 36353 entries
drop colummns: 18 columns
 After droping columns : 11 columns
Cleaning duplicates 'nom', 'type', and 'siren': 36353 entries
After removing duplicates: 36347 entries


In [29]:
communities_clean.shape

(36347, 11)

### Enrichir les données

In [30]:
communities_final = communities_clean.copy()

#### 1. Latitude longitude
**source** : selected_communities_data.csv

In [31]:
selected_communities = pd.read_csv(path + 'selected_communities_data.csv', delimiter=';')

In [32]:
selected_communities.head()

,Unnamed: 0,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,url_ptf,url_datagouv,id_datagouv,merge,ptf,trancheeffectifsunitelegale,effectifssup50,longitude,latitude
0,0,Guadeloupe,239710015,REG,1,NaN,NaN,NaN,NaN,396153.0,NaN,https://regionguadeloupe.opendatasoft.com/expl...,NaN,NaN,ptf,KaruData,42.0,True,-61.568686,16.252882
1,1,Île-de-France,237500079,REG,11,NaN,NaN,NaN,NaN,12291279.0,NaN,https://data.iledefrance.fr/explore/?refine.pu...,https://www.data.gouv.fr/fr/organizations/534f...,534fffa8a3a7292c64a780c8,ptf+datagouv,"Ile-de-France, DataGouv",53.0,True,2.504722,48.709167
2,2,Martinique,200055507,CTU,2,NaN,NaN,NaN,NaN,377711.0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,True,-61.015827,14.636792
3,3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,NaN,NaN,2634852.0,NaN,https://data.centrevaldeloire.fr/explore/?sort...,https://www.data.gouv.fr/fr/organizations/regi...,561a7f00c751df4e6dcdbb48,ptf+datagouv,"Centre Val-de-Loire, DataGouv",51.0,True,1.685278,47.480556
4,4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,NaN,NaN,2885864.0,NaN,https://trouver.ternum-bfc.fr/organization/reg...,https://www.data.gouv.fr/fr/organizations/5835...,5835cdb988ee383e0bc65bb3,ptf+datagouv,"Bourgogne-Franche-Comté, DataGouv",51.0,True,4.809167,47.235278


In [33]:
selected_communities["siren"]= selected_communities["siren"].apply(clean_numeros)

In [34]:
communities_final = communities_final.merge(selected_communities[['siren', 'type', 'longitude','latitude']],on=['siren', 'type'], how='left' )

In [35]:
communities_final[communities_final["latitude"].isna()]['type'].value_counts()

COM    31841
CC         1
Name: type, dtype: int64

* **Sources** : 
    * 20230823-communes-departement-region.csv
    * banatic-sireninsee-last.csv

In [36]:
com_dep_reg_df = pd.read_csv(path + '20230823-communes-departement-region.csv')

In [37]:
com_dep_reg_df.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [38]:
com_dep_reg_df['insee'] = com_dep_reg_df['code_commune_INSEE']

In [39]:
siren_insee = pd.read_csv(path + 'banatic-sireninsee-last.csv', delimiter=';', encoding='ISO-8859-1')

In [40]:
siren_insee.head()

,Reg_com,dep_com,siren,insee,nom_com,ptot_2024,pmun_2024,pcap_2024
0,84,01,210100012,01001,L'Abergement-Clémenciat,848,832,16
1,84,01,210100020,01002,L'Abergement-de-Varey,273,267,6
2,84,01,210100046,01004,Ambérieu-en-Bugey,15 240,14 854,386
3,84,01,210100053,01005,Ambérieux-en-Dombes,1 921,1 897,24
4,84,01,210100061,01006,Ambléon,113,113,0


In [41]:
com_dep_reg_df = com_dep_reg_df.merge(siren_insee[['siren', 'insee']], on=['insee'], how='left')

In [42]:
com_dep_reg_df.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 35944
latitude                  269
longitude                 269
code_commune                7
article                 36621
nom_commune                 0
nom_commune_complet         0
code_departement            7
nom_departement           267
code_region               267
nom_region                267
insee                       0
siren                    4707
dtype: int64

In [43]:
com_dep_reg_df.dropna(subset='siren', axis=0, inplace=True)

In [44]:
com_dep_reg_df["siren"] = com_dep_reg_df['siren'].apply(clean_numeros)

In [45]:

valeurs_dict_la = com_dep_reg_df.groupby('siren')['latitude'].first().to_dict()
valeurs_dict_lg = com_dep_reg_df.groupby('siren')['longitude'].first().to_dict()
communities_final['latitude'] = communities_final['latitude'].fillna(communities_final['siren'].map(valeurs_dict_la))
communities_final['longitude'] =communities_final['longitude'].fillna(communities_final['siren'].map(valeurs_dict_lg))


In [46]:
communities_final.isna().sum()

nom                               2
siren                             0
type                              1
cog                            1268
code_dept                       117
code_departement_3digits        117
code_region                      19
population                        1
epci                           1429
trancheeffectifsunitelegale    2109
effectifssup50                    0
longitude                      3059
latitude                       3059
dtype: int64

In [47]:
# check vlue null of nom
communities_final[communities_final['nom'].isna()]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,longitude,latitude
10002,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,1.295948,49.20122
36350,NaN,0,NaN,NaN,NaN,NaN,NaN,37323.0,NaN,NaN,False,NaN,NaN


#### Densite et superficie 
**source**: communes-france-2025.csv

In [48]:
commune_france = pd.read_csv(path + 'communes-france-2025.csv',low_memory=False)

In [49]:
commune_france.columns

Index(['Unnamed: 0', 'code_insee', 'nom_standard', 'nom_sans_pronom', 'nom_a',
       'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'typecom',
       'typecom_texte', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom',
       'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code',
       'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi',
       'code_insee_centre_zone_emploi', 'code_unite_urbaine',
       'nom_unite_urbaine', 'taille_unite_urbaine',
       'type_commune_unite_urbaine', 'statut_commune_unite_urbaine',
       'population', 'superficie_hectare', 'superficie_km2', 'densite',
       'altitude_moyenne', 'altitude_minimale', 'altitude_maximale',
       'latitude_mairie', 'longitude_mairie', 'latitude_centre',
       'longitude_centre', 'grille_densite', 'grille_densite_texte',
       'niveau_equipements_services', 'niveau_equipements_services_texte',
       'gentile', 'url_wikipedia', 'url_villedereve'],
      dtype='object')

In [50]:
commune_france['typecom'].value_counts()

COM    34935
Name: typecom, dtype: int64

In [51]:
#ajouter le siren
commune_france['insee'] = commune_france['code_insee']
commune_france = commune_france.merge(siren_insee[['siren', 'insee']], on='insee', how = 'left')

In [52]:
commune_france.isna().sum()

Unnamed: 0                               0
code_insee                               0
nom_standard                             0
nom_sans_pronom                          0
nom_a                                    0
nom_de                                   0
nom_sans_accent                          0
nom_standard_majuscule                   0
typecom                                  0
typecom_texte                            0
reg_code                                 0
reg_nom                                  0
dep_code                                 0
dep_nom                                  0
canton_code                             56
canton_nom                              56
epci_code                                3
epci_nom                                 3
academie_code                            0
academie_nom                             0
code_postal                              3
codes_postaux                            3
zone_emploi                              3
code_insee_

In [53]:
commune_france['type']=commune_france['typecom']
commune_france['siren'] = commune_france['siren'].apply(clean_numeros)

In [54]:
communities_final = communities_final.merge(commune_france[['siren', 'type', 'superficie_km2','densite']], on=['siren', 'type'], how='left')

In [55]:
communities_final.isna().sum()

nom                               2
siren                             0
type                              1
cog                            1268
code_dept                       117
code_departement_3digits        117
code_region                      19
population                        1
epci                           1429
trancheeffectifsunitelegale    2109
effectifssup50                    0
longitude                      3059
latitude                       3059
superficie_km2                 1449
densite                        1450
dtype: int64

In [56]:
communities_final.head()

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,longitude,latitude,superficie_km2,densite
0,Guadeloupe,239710015,REG,1,NaN,NaN,NaN,396153.0,NaN,42.0,True,-61.568686,16.252882,NaN,NaN
1,Île-de-France,237500079,REG,11,NaN,NaN,NaN,12291279.0,NaN,53.0,True,2.504722,48.709167,NaN,NaN
2,Martinique,200055507,CTU,2,NaN,NaN,NaN,377711.0,NaN,51.0,True,-61.015827,14.636792,NaN,NaN
3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,NaN,2634852.0,NaN,51.0,True,1.685278,47.480556,NaN,NaN
4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,NaN,2885864.0,NaN,51.0,True,4.809167,47.235278,NaN,NaN


#### Les élus

In [57]:
# TODO

#### Obligation publication

In [58]:
## TODO

#### Compléter les codes départements pour faire le lien  avec les communes ou autres collectivités
source nocodb :ofgl-base-departements-consolidee.csv

In [59]:
bases_departements = pd.read_csv(path +'ofgl-base-departements-consolidee.csv', delimiter=';')

In [60]:
bases_departements['siren'] = bases_departements['siren'].apply(clean_numeros)
bases_departements['dep_code'] = bases_departements['dep_code'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)

In [61]:
valeurs_dict_dep = bases_departements.groupby('siren')['dep_code'].first().to_dict()
communities_final['code_dept'] = communities_final['code_dept'].fillna(communities_final['siren'].map(valeurs_dict_dep))

In [62]:
communities_final[communities_final["code_dept"].isnull()]["type"].value_counts()

REG    14
CTU     3
COM     1
Name: type, dtype: int64

In [63]:
communities_final[(communities_final["code_dept"].isnull()) & (communities_final["type"]=='COM')]


,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,longitude,latitude,superficie_km2,densite
10002,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,1.295948,49.20122,15.0,117.0


##### Remarque: une seule commune avec nom null et code dept null: siren: 200063782

#### Compléter les codes régions pour faire le lien  avec les communes, départements ou autres collectivités
Sources: NocoDB ofgl-base-regions-consolidee.csv 

In [64]:
communities_final[communities_final['code_region'].isna()]["type"].value_counts()

REG    14
CTU     3
COM     1
Name: type, dtype: int64

In [65]:
base_regions = pd.read_csv(path + 'ofgl-base-regions-consolidee.csv', delimiter=';')

In [66]:
base_regions['siren'] = base_regions['siren'].apply(clean_numeros)
base_regions['reg_code'] = base_regions['reg_code'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)

In [67]:
valeurs_dict_reg = base_regions.groupby('siren')['reg_code'].first().to_dict()
communities_final['code_region'] = communities_final['code_region'].fillna(communities_final['siren'].map(valeurs_dict_reg))

In [68]:
communities_final[communities_final['code_region'].isna()]["type"].value_counts()

COM    1
Name: type, dtype: int64

Remarque: la même commune avec code région null: 200063782